In [4]:
# !pip install pandas
# !pip install scikit-learn

In [5]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Mengatur seed untuk reproduksibilitas
seed = 42

# Load dataset iris
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = iris.target

# Mengacak dataset
iris_df = iris_df.sample(frac=1, random_state=seed)

# Memilih fitur dan target
X = iris_df.drop(columns=['species'])
y = iris_df['species']

# Memisahkan data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=seed, stratify=y
)

# Membuat instance dari RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=seed)

# Melatih classifier pada data pelatihan
clf.fit(X_train, y_train)

# Memprediksi pada set pengujian
y_pred = clf.predict(X_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")  # Output: Accuracy: 0.98


Accuracy: 0.96


In [11]:
y.drop_duplicates()

73     1
18     0
118    2
Name: species, dtype: int64

In [7]:
import joblib

# save the model to disk
joblib.dump(clf, "rf_model.sav")

['rf_model.sav']

In [8]:
%%writefile prediction.py
import joblib

def predict(data):
    clf = joblib.load("rf_model.sav")
    return clf.predict(data)

Writing prediction.py


In [13]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import numpy as np
from prediction import predict

species_mapping = {0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'}

st.title('Classifying Iris Flowers')
st.markdown('Toy model to play to classify iris flowers into \
     (setosa, versicolor, virginica) based on their sepal/petal \
    and length/width.')

st.header("Plant Features")
col1, col2 = st.columns(2)

with col1:
    st.text("Sepal characteristics")
    sepal_l = st.slider('Sepal lenght (cm)', 1.0, 8.0, 0.5)
    sepal_w = st.slider('Sepal width (cm)', 2.0, 4.4, 0.5)

with col2:
    st.text("Petal characteristics")
    petal_l = st.slider('Petal lenght (cm)', 1.0, 7.0, 0.5)
    petal_w = st.slider('Petal width (cm)', 0.1, 2.5, 0.5)

st.text('')
if st.button("Predict type of Iris"):
    result = predict(np.array([[sepal_l, sepal_w, petal_l, petal_w]]))
    species = species_mapping.get(result[0], "Unknown")
    st.text(f"The predicted species is: {species}")

st.text('')

Overwriting streamlit_app.py
